**Python notebook to illustrate capabilities of ARM JupterLab + [HPC platform](https://www.arm.gov/capabilities/computing-resources).**
1. Use ARM Live Web Service for data download
2. Generate ARM cloud classification [(CLDTYPE) VAP](https://www.arm.gov/capabilities/vaps/cldtype) product

This notebook is not a complete implementation of the [CLDTYPE VAP](https://www.arm.gov/capabilities/vaps/cldtype), rather a simplfied implementation for purpose of demonstration.

Author: *Jitendra (Jitu) Kumar, Oak Ridge National Laboratory*

In [ ]:
# Add ARM Live credentials here
user = "User.Name"
credential = "add your web key here"

# Get your credentials at https://adc.arm.gov/armlive/

In [ ]:
import sys
import glob
import time

import numpy as np
from netCDF4 import Dataset
import logging
import csv
import glob

In [ ]:
# Routine for ARM Live Web Service
from urllib import request
import json
import os


def get_files(user: str, credential: str, datastream: str, start: str, end: str, output=None):

    # combine user and credential
    username = "{}:{}".format(user, credential)

    # start and end strings for query_url are constructed
    if start: start = "&start={}".format(start)
    if end: end = "&end={}".format(end)

    # build link, user = username:credentials, datastream can be partial, start and end in form YYYY-MM-DD
    query_url = 'https://adc.arm.gov/armlive/livedata/query?user={0}&ds={1}{2}{3}&wt=json'\
        .format(username, datastream, start, end)
    #print("query url = {}".format(query_url))

    # get url response, read the body of the message, and decode from bytes type to utf-8 string
    response_body = request.urlopen(query_url).read().decode("utf-8")
    # if the response is an html doc, then there was an error with the user
    if response_body[1:14] == "!DOCTYPE html":
        print("Error with user. Check username or token.")
        exit(1)

    # parse into json object
    response_body_json = json.loads(response_body)
    #print("response body:\n{0}\n".format(json.dumps(response_body_json, indent=True)))

    # construct output directory
    if output:
        # output files to directory specified
        output_dir = os.path.join(output)
    else:
        # if no folder given, add datastream folder to current working dir to prevent file mix-up
        output_dir = os.path.join(os.getcwd(), datastream)

    # response is successful and files were returned
    if response_body_json["status"] == "success" and len(response_body_json["files"]) > 0:
        for f in response_body_json['files']:
            print("[DOWNLOADING] {}".format(f))
            # construct link to web service saveData function
            save_data_url = "https://adc.arm.gov/armlive/livedata/saveData?user={0}&file={1}"\
                .format(username, f)
            #print("downloading file: {0}\n\tusing link: {1}".format(f, save_data_url))

            output_file = os.path.join(output_dir, f)
            # make directory if it doesn't exist
            if not os.path.isdir(output_dir):
                os.makedirs(output_dir)
            # create file and write bytes to file
            with open(output_file, 'wb') as open_bytes_file:
                open_bytes_file.write(request.urlopen(save_data_url).read())
            #print("file saved to --> {}\n".format(output_file))
    else:
        print("No files returned or url status error.\nCheck datastream name, start, and end date.")
    print "DOWNLOAD COMPLETE\n"

In [ ]:
# Download the datasets required for CLDTYPE VAP
# Download arscl data
ds = "nsaarscl1clothC1.c1"
start = "2010-03-01"
end = "2010-03-05"
out="live_download_dir"

get_files(user, credential, ds, start, end, out)

# Download met data
ds = "nsametC1.b1"
start = "2010-03-01"
end = "2010-03-05"
out = "live_download_dir"
get_files(user, credential, ds, start, end, out)

In [ ]:
# Routines for cloud classification
class create_time_series:
    '''
    Class to load staged data from ARM database, to parse and analyze it
    and save the results as NetCDF file.

    Args:
        None.

    Returns:
        None. Saves NetCDF data file.
    '''

    def __init__(self, filename):
        '''Initializer for create_time_series class

        Args:
            filename (str): name for NetCDF output file

        Returns:
            None.
        '''

        # initialize self object
        self.filename = filename
        print("Working on file: %s"%(self.filename))
        


    def rain_check(self, rain_rate):
        '''
        Checks the rate of rainfall to see if instrument
        measurements are accurate

        Args:
            rain_rate (float): rainfall rate (mm/hr)

        Returns:
            rain_bool (bool): boolean determining if instrument
                              measurements are accurate
        '''

        # check to see if it is rainging to much
        if rain_rate >= 1:
            rain_bool = True
            return rain_bool

        # else, it is not raining too much
        elif rain_rate < 1:
            rain_bool = False
            return rain_bool


    def load_cloud_data(self, file):
        '''
        Loads data about clouds from NetCDF file

        Args:
            file (str): file name for data to load

        Returns:
            time_array (np array): time array of interest
            cloud_base_data (np array): raw cloud base measurements
            cloud_top_data (np array): raw cloud top measurements
        '''

        # load cloud data
        rootgrp = Dataset(file, "r", format="NETCDF4")

        # extract cloud data
        time_array = rootgrp.variables['time_offset'][:]
        cloud_base_data = rootgrp.variables['CloudLayerBottomHeightMplZwang'][:]
        cloud_top_data = rootgrp.variables['CloudLayerTopHeightMplZwang'][:]

        # create dataset, start writing NetCDF file
        rg = Dataset(self.filename, "w", format="NETCDF4")

        # create base_time
        base_time = rg.createDimension('base_time', 1)
        bt = rg.createVariable('base_time', 'i4', ('base_time',))
        bt[0] = rootgrp.variables['base_time'][:]
        bt.units = 'seconds since 1970-1-1 0:00:00 0:00'
        bt.long_name = 'Base time in Epoch'
        bt.ancillary_variables = 'time_offset'

        # create height variable
        height = rg.createDimension('height', len(rootgrp['Heights']))
        h = rg.createVariable('height', 'f8', ('height',))
        h[:] = rootgrp['Heights'][:]
        h.units = 'm'
        h.long_name = 'Height above ground level'
        h.standard_name = 'height'

        rg.close()

        # close cloud data set
        rootgrp.close()

        return time_array, cloud_base_data, cloud_top_data

    def load_rain_data(self, rain_file):
        '''
        Loads data from NetCDF file

        Args:
            rain_file (str): file name for rain data to load

        Returns:
            rain_array (np array): rainfall measurements (mm/hr)
        '''

        # load data
        rootgrp = Dataset(rain_file, 'r', format='NETCDF4')

        # extract data
        rain_array = rootgrp.variables['pws_precip_rate_mean_1min'][:]

        rootgrp.close()

        return rain_array

    def classify_cloud(self, cld_bot, cld_top, cld_thick, rn_rate):
        '''
        Classify the type of cloud given.

        Args:
            cld_bot (float): Height of the bottom of a cloud
            cld_top (float): Height of the top of a cloud
            cld_thick (float): Thickness of a cloud
            rn_rate (float): rainfall rate during measurement

        Returns:
            cld_type (int): cloud type represented by an integer
        '''

        # check for no clouds
        if (cld_bot == 0 and cld_top == 0):
            cld_type = -9999

        elif self.rain_check(rn_rate):
            cld_type = -9999

        # Identify low level clouds
        elif cld_bot < 4000:

            # Low clouds
            if cld_top < 4000:
                cld_type = 1

            # Congestus clouds
            elif 4000 <= cld_top <= 8000:
                cld_type = 2

            # Deep convection clouds
            elif cld_top > 8000:
                cld_type = 3

        # Medium height clouds
        elif 4000 <= cld_bot <= 8000:

            # Altocumulus clouds
            if ((4000 <= cld_top <= 8000) and cld_thick < 1500):
                cld_type = 4

            # Altostratus clouds
            elif ((4000 <= cld_top <= 8000) and cld_thick >= 1500):
                cld_type = 5

            # Cirrostratus/Anvil clouds
            elif (cld_top > 8000):
                cld_type = 6

        # Cirrus clouds
        elif (cld_bot > 8000):
            cld_type = 7

        return cld_type

    def analyze_data(self, cloud_base_data, cloud_top_data, rain_rate, time_array):
        '''
        Analyzes cloud data and determines what types of cloud there are

        Args:
            cloud_base_data (np array): raw cloud base height
            cloud_top_data (np array): raw cloud top height
            rain_rate (float): rainfall rate (mm/hr)
            time_array (np array): time array of interest

        Returns:
            cloud_layer_array (np array): classification of cloud layers
            time_bounds_array (np array): time bounds array
            new_time_array (np array): new array for time, after averaging over 1 minute
        '''

        # create empty array cloud layer classification
        cloud_layer_array = np.zeros((int((len(cloud_base_data)/6)), 10))

        # create time bounds array
        time_bounds_array = np.zeros((int((len(cloud_base_data)/6)), 2))
        new_time_array = np.zeros((int((len(cloud_base_data)/6))))

        # loop over all data
        for i in range(0, 1440):

            # update time bounds
            time_bounds_array[i, 0] = i
            time_bounds_array[i, 1] = (i+1)

            # update new time array in minutes
            new_time_array[i] = i

            for j in range(0, 10):

                cld_type_list = []

                # iterate over 1 minute interval to find most occuring
                # cloud type
                for k in range(0,6):

                    # get cloud top and bottom and thickness
                    cld_bot = cloud_base_data[(i*6+k), j]
                    cld_top = cloud_top_data[(i*6+k), j]
                    cld_thick = cld_top - cld_bot

                    # get rain rate
                    rn_rate = rain_rate[i]

                    # clasifyl cloud
                    cld_type = self.classify_cloud(cld_bot, cld_top, cld_thick, rn_rate)

                    # add cloud type to list
                    cld_type_list.append(cld_type)

                # find most frequently occuring cloud type at cloud level
                cld_type = max(set(cld_type_list), key=cld_type_list.count)

                # update cloud type in array
                cloud_layer_array[i, j] = cld_type

        return cloud_layer_array, time_bounds_array, new_time_array

    def save_cloud_data(self, new_time_array, time_bounds_array, cloud_layer_array, cloud_base_data):
        '''
        Creates NetCDF file from time and cloud layer data

        Args:
            new_time_array (np array): time array of interest
            time_bounds_array (np array): time bounds array
            cloud_layers (np array): classification of cloud layers
            cloud_base_data (np array): raw cloud base height

        Returns:
            Mone. Saves NetCDF file.
        '''

        # create dataset, start writing NetCDF file
        rootgrp = Dataset(self.filename, "a", format="NETCDF4")

        # create dimensions for NetCDF file
        time_offset = rootgrp.createDimension('time_offset', len(new_time_array))
        time = rootgrp.createDimension('time', len(new_time_array))
        cloudtype = rootgrp.createDimension('cloudtype', len(cloud_layer_array))
        layer = rootgrp.createDimension('layer', 10)

        # create time_offset variable
        t = rootgrp.createVariable('time_offset', 'f8', ('time_offset',))
        t.long_name = 'Time Offset from base_time'
        t.units = 'minutes'
        t.ancillary_variables = 'base_time'
        t[:] = new_time_array[:]

        # create time variable
        t_m = rootgrp.createVariable('time', 'f8', ('time',))
        t_m.long_name = 'Time Offset from midnight'
        t_m.units = 'minutes'
        t_m.ancillary_variables = 'time_bounds'
        t_m[:] = new_time_array[:]

        # create time bounds variable
        time_bounds = rootgrp.createDimension('time_bounds', 2)
        tb = rootgrp.createVariable('time_bounds', 'f8', ('time_offset', 'time_bounds'))
        tb[:] = time_bounds_array[:]
        tb.long_name = 'Time cell bounds'
        tb.bound_offsets = -30, 30

        # create layer variable
        l = rootgrp.createVariable('layer', 'i4', ('layer'))
        l.long_name = 'Cloud layer number'
        l.units = 'unitless'
        l[:] = np.arange(1,11,1)

        # create cloud type variable
        cld_layer = rootgrp.createVariable('cloudtype', 'i4', ('cloudtype', 'layer'))
        cld_layer[:] = cloud_layer_array[:]
        cld_layer.long_name = 'Cloud type'
        cld_layer.units = 'unitless'
        cld_layer.ancillary_variables = 'qc_cloudtype'
        cld_layer.missing_value = -9999
        cld_layer.flag_values = 1, 2, 3, 4, 5, 6, 7
        cld_layer.flag_meanings = 'low_cloud', 'congestus', 'deep_convection', 'altocumulus', 'altostratus', 'cirrostratus/anvil', 'cirrus'

        return


    def cloud_analysis(self, file, rain_file):
        '''Main algrithm, loads cloud data and analyzes it. Returns data as a NetCDF file.

        Args:
            file (str): file name for data to load
            rain_file (str): file name for rain data to load

        Returns:
            None. Saves data to NetCDF file.
        '''

        # split to allow it to load from DB

        # load cloud data
        time_array, cloud_base_data, cloud_top_data = self.load_cloud_data(file)

        # load rain data
        rain_rate = self.load_rain_data(rain_file)


        # separate time series creation and netCDF creation
        # use in memory data structure for manipulation of data
        # make data file (netCDF, CSV,...) afterwards

        # analyze cloud data
        cloud_layer_array, time_bounds_array, new_time_array = self.analyze_data(cloud_base_data, cloud_top_data, rain_rate, time_array)

        # save cloud data
        self.save_cloud_data(new_time_array, time_bounds_array, cloud_layer_array, cloud_base_data)

        return

In [ ]:
# Run CLDTYPE VAP for downloaded data
directory='live_download_dir'

file_list = glob.glob(directory + "/*arscl*.cdf")
num_files = len(file_list)

# run cloud classification on each file
for f in range(num_files):
    filename=file_list[f]
    split_file = filename.split('/')
    arscl_file = split_file[-1]
    met_file = directory + '/' + arscl_file[0:3] + 'metC1.b1' + arscl_file[-20:]
    cc_file = directory + '/nsa_C1_cloud_classification' + arscl_file[-20:-11] + '.cdf'

    ts = create_time_series(filename=cc_file)
    ts.cloud_analysis(file=filename, rain_file=met_file)
print "COMPLETE: Cloud Classification VAP"